Import bibliotek

In [7]:
import string
import nltk
import json
import numpy as np
import pandas as pd
import itertools as it
import operator as op

from nltk import word_tokenize
from nltk.corpus import PlaintextCorpusReader
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

# Korpus dokumentów

In [8]:
corpus_dir = "./streszczenie_original"
corpus = PlaintextCorpusReader(corpus_dir, ".*\\.txt")
files_names = corpus.fileids()
files_names

['Harry Potter i Czara Ognia.txt',
 'Harry Potter i Insygnia Smierci.txt',
 'Harry Potter i Kamien Filozoficzny.txt',
 'Harry Potter i Komnata Tajemnic.txt',
 'Harry Potter i Ksiaze Polkrwi.txt',
 'Harry Potter i Przeklete Dziecko.txt',
 'Harry Potter i Wiezien Azkabanu.txt',
 'Harry Potter i Zakon Feniksa.txt',
 'Opowiesci z Narni. Kon i jego chlopiec.txt',
 'Opowiesci z Narni. Ksiaze Kaspian.txt',
 'Opowiesci z Narni. Lew, czarownica i stara szafa.txt',
 'Opowiesci z Narni. Ostatnia bitwa.txt',
 'Opowiesci z Narni. Podroz Wedrowca do Switu.txt',
 'Opowiesci z Narni. Siostrzeniec Czarodzieja.txt',
 'Opowiesci z Narni. Srebrne krzeslo.txt',
 'Zmierzch.Ksiezyc w Nowiu.txt',
 'Zmierzch.Przed Switem.txt',
 'Zmierzch.Zacmienie.txt',
 'Zmierzch.Zmierzch.txt',
 'Zmierzch.Zycie i Smierc.txt']

# Wstępne przygotowanie dokumentów

In [9]:
documents= {}
for file in files_names:
    documents[file] = corpus.raw(file)
print(json.dumps(documents, indent=4, ensure_ascii=False))

{
    "Harry Potter i Czara Ognia.txt": "Lord Voldemort, wraz ze swoim sługą Glizdogonem przybywa do domu należącego niegdyś do rodziny Riddle'ów położonego w Little Hangleton. Tam układają plan dotyczący zamordowania Harry'ego Pottera, co ma pomóc Czarnemu Panu w odzyskaniu jego dawnej mocy. Jest także mowa o wiernym słudze Czarnego Pana, który wkrótce ma znaleźć się w Hogwarcie. Wszystko to słyszy mieszkający nieopodal ogrodnik, Frank Bryce. Voldemort odkrywając, iż mężczyzna podsłuchuje, zaprasza go do środka i uśmierca. Harry budzi się z przerażającego snu. Jego blizna pulsuje okropnym bólem. Próbuje sobie przypomnieć o czym śnił, a z pamięci wyłania się obraz mrocznego pokoju, węża leżącego przed kominkiem oraz Glizdogona i Lorda Voldemorta planujących morderstwo. Harry zastanawia się czy powinien komuś powiedzieć o problemach z blizną, ponieważ, gdy bolała go ostatnim razem, Voldemort wdarł się do Hogwartu. Ostatecznie postanawia napisać do Syriusza i zejść na śniadanie. W trakci

In [10]:
stoplist_file = open("./stopwords_pl.txt", "r", encoding="utf-8")
stoplist = stoplist_file.read()
stoplist_file.close()
stoplist = stoplist[4:]
stoplist

'is file was created from the carrot2 project and is distributed under the BSD license.\n# See http://project.carrot2.org/license.html\n# Also see http://www.opensource.org/licenses/bsd-license.html\n# From trunk/core/carrot2-util-text/src-resources/stopwords.pl\nvol\no.o.\nmgr\ngodz\nzł\nwww\npl\nul\ntel\nhab\nprof\ninż\ndr\ni\nu\naby\nalbo\nale\nani\naż\nbardzo\nbez\nbo\nbowiem\nby\nbyli\nbym\nbył\nbyła\nbyło\nbyły\nbyć\nbędzie\nbędą\nchce\nchoć\nco\ncoraz\ncoś\nczy\nczyli\nczęsto\ndla\ndo\ngdy\ngdyby\ngdyż\ngdzie\ngo\nich\nim\ninne\niż\nja\njak\njakie\njako\nje\njednak\njednym\njedynie\njego\njej\njest\njeszcze\njeśli\njeżeli\njuż\nją\nkiedy\nkilku\nkto\nktóra\nktóre\nktórego\nktórej\nktóry\nktórych\nktórym\nktórzy\nlat\nlecz\nlub\nma\nmają\nmamy\nmi\nmiał\nmimo\nmnie\nmogą\nmoże\nmożna\nmu\nmusi\nna\nnad\nnam\nnas\nnawet\nnic\nnich\nnie\nniej\nnim\nniż\nno\nnowe\nnp\nnr\no\nod\nok\non\none\noraz\npan\npo\npod\nponad\nponieważ\npoza\nprzed\nprzede\nprzez\nprzy\nraz\nrazie\nroku\nrów

In [11]:
# Pomijamy na macoes -> brak pakietu morfeusz2
# import morfeusz2
#
# def lemmatize(text):
#     morf = morfeusz2.Morfeusz()
#     segments = it.groupby(morf.analyse(text))
#     def disambiguate(group):
#         pairs = ((len(descr), lemma) for _, _, (_, lemma, descr, _, _,) in group)
#         perpl, lemma = min(pairs)
#         return lemma.split(":")[0]
#     lemmas = (disambiguate(group) for key, group in segments)
#     return " ".join(filter(str.isalpha, lemmas))

In [12]:
for key in documents:
    documents[key] = documents[key].lower()
    documents[key] = "".join([char for char in documents[key] if char not in string.punctuation])
    # documents[key] = lemmatize(documents[key]) # wylaczone na macoes
    documents[key] = " ".join([word for word in word_tokenize(documents[key], language='polish') if word not in stoplist])
print(json.dumps(documents, indent=4, ensure_ascii=False))

{
    "Harry Potter i Czara Ognia.txt": "lord voldemort wraz swoim sługą glizdogonem przybywa domu należącego niegdyś rodziny riddleów położonego little hangleton układają plan dotyczący zamordowania harryego pottera pomóc czarnemu panu odzyskaniu dawnej mocy mowa wiernym słudze czarnego pana wkrótce znaleźć hogwarcie słyszy mieszkający nieopodal ogrodnik frank bryce voldemort odkrywając mężczyzna podsłuchuje zaprasza środka uśmierca harry budzi przerażającego snu blizna pulsuje okropnym bólem próbuje przypomnieć czym śnił pamięci wyłania obraz mrocznego pokoju węża leżącego kominkiem glizdogona lorda voldemorta planujących morderstwo harry zastanawia powinien komuś powiedzieć problemach blizną bolała ostatnim razem voldemort wdarł hogwartu ostatecznie postanawia napisać syriusza zejść śniadanie trakcie śniadania wuj vernon otrzymuje list weasleyowie oferują zabranie harryego finał mistrzostw świata quidditchu ugoszczenie resztę wakacji długim namyśle użyciu harryego odpowiednich argum

# Utworzenie macierzy częstości

In [13]:
docs = pd.DataFrame().from_dict(documents, orient='index')
docs.columns = ['content']
docs


,content
Harry Potter i Czara Ognia.txt,lord voldemort wraz swoim sługą glizdogonem pr...
Harry Potter i Insygnia Smierci.txt,podczas zebrania śmierciożerców odbywa dworze ...
Harry Potter i Kamien Filozoficzny.txt,ulicy privet drive pojawia stary człowiek dług...
Harry Potter i Komnata Tajemnic.txt,wuj vernon zły hedwiga robi bałagan domu tymcz...
Harry Potter i Ksiaze Polkrwi.txt,mugolskiego premiera odwiedza korneliusz knot ...
Harry Potter i Przeklete Dziecko.txt,scena pierwsza druga opowiada historię została...
Harry Potter i Wiezien Azkabanu.txt,harry leży kocami łóżku odrabia prace domowe z...
Harry Potter i Zakon Feniksa.txt,dudley zły wyżywa harrym dudley zostaje zaatak...
Opowiesci z Narni. Kon i jego chlopiec.txt,akcja książki rozgrywa narnii czasów panowania...
Opowiesci z Narni. Ksiaze Kaspian.txt,mija wydarzeń opisanych powieści lew czarownic...


In [14]:
count_vertorizer = CountVectorizer()
counts_tf = count_vertorizer.fit_transform(docs['content'])
counts_tf.toarray()


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(20, 7334))

In [15]:
tfidf_vectorizer = TfidfVectorizer()
counts_tfidf = tfidf_vectorizer.fit_transform(docs['content'])
counts_tfidf.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.02007235, 0.02283504, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], shape=(20, 7334))